# Practical 6: Mapping and visualisations

In this practical we will focus on consolidating some of the lessons learned on creating effective maps. In the lecture we discussed some of the important design considerations for making clear, accessible, legible, *honest* maps. Now we implement a few of those technical points. We've already been making quick plots of our geographic data and in the last practical we started to create *choropleth* maps, but we'll look at a few more options for mapping.

This practical covers several topics (and a few smaller ones along the way):
* Mapping with point-referenced data
* Adding in external basemaps or satellite imagery
* Creating thematic maps
* Adjusting classifications visualising data
* Controlling multiple layers and multiple maps

Creating static maps in Python with `GeoPandas` relies on `matplotlib` extensively. If you're less familiar with this graphics library, check out some of the tutorials: [https://matplotlib.org/tutorials/index.html](https://matplotlib.org/tutorials/index.html). It's also good to refer back to the documentation occassionally because there are many things you can control/adjust in a plot, and we can't show you everything in this limited practical session.

Be sure to read the code blocks and the comments carefully in this practical. There are a lot of parameters and settings. Experiment with changing them and re-mapping your data.

In [ ]:
# load packages
import mapclassify
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd

In [ ]:
# use the magic function
# https://ipython.readthedocs.io/en/stable/interactive/plotting.html
%matplotlib inline

## Mapping with Point data

We will start by working with some georeferenced point data for Southampton.

### Load the data

This practical is going to re-use some of the data you created in Practical 5 when calculating the distance between LSOA and supermarket in Southampton (`../data/soton_lsoa_distance.gpkg`). If you don't have that file saved, then go back to the last practical and recreate it and save a local copy before continuing.

In [ ]:
# load fast food restaurant locations
ff = gpd.read_file('../data/soton_fastfood.gpkg')

In [ ]:
# convert from multipoint to single
ff = ff.explode()

In [ ]:
ff.head()

In [ ]:
# load supermarket locations
# these are the data that we cleaned and prepared in the last practical
smkt = gpd.read_file('../data/soton_smkts.gpkg')

In [ ]:
# convert from multipoint to single
smkt = smkt.explode()

In [ ]:
smkt.head()

### Basic plotting

As we have seen previously, we can use `.plot()` with `GeoDataFrames` to easily create a map. This method specifically creates a `matplotlib` axes object, which gives us a lot of flexibility for modifying the map.

In [ ]:
# create an axes object
# based on plotting the fast food locations
ax = ff.plot()

# add supermarket locations
smkt.plot(ax=ax)

Conveniently, the two layers have different colours by default.

In [ ]:
# here we can see what we created is a matplotlib object
type(ax)

We can plot these layers on separate axes. This technique can help us create the effect of *small multiples* where different cuts of the data are visualised next to one another. 

In [ ]:
# create a figure with two subplots
# layers are plotted separately
fig, (ax1, ax2) = plt.subplots(ncols=2,
                               sharex=True, # make both plots line up
                               sharey=True)

# plot fast food
ff.plot(ax=ax1)
# plot supermarkets
smkt.plot(ax=ax2)

If we want to adjust the figure size, an easy way is to modify the "subplot" which will contain one figure (with multiple layers). The figure size is specified in (width, height) dimensions.

In [ ]:
# adjusting figure size
fig, ax = plt.subplots(figsize=(15, 10))

# plot fast food locations
ff.plot(ax=ax)

# overlay supermarket locations
smkt.plot(ax=ax)

By plotting these two datasets separately, we can control each layer's display properties. Let's change the colour and the shape used for plotting.

The list of potential point markers is here: [https://matplotlib.org/3.2.1/api/markers_api.html](https://matplotlib.org/3.2.1/api/markers_api.html).

In [ ]:
# adjusting layer colour and symbol style
fig, ax = plt.subplots(figsize=(15, 10))

# plot fast food locations
ff.plot(ax=ax,
        color='purple')

# overlay supermarket locations
smkt.plot(ax=ax,
          color='green',
          marker='^')

### Colour by attribute

Sometimes we may not have separate layers, but a combined data layer and we want to adjust the *symbology* based on a value in the dataset. Rather than specifying the parameters manually as we have been, we can always use an attribute within the dataset, but we have to *map* that attribute to a set of colours. 

As we discussed, choice of colour can be very important in cartographic design. It can convey certain properties making your map more legible (or more confusing!). It's also important to consider accessibility in your maps. How would a person who is colourblind interpret your map? Or if you map is printed in black and white? May be it's best to combine different symbol style and colours.

A great resource to explore different colour combinations is [ColourBrewer](https://colorbrewer2.org).

Let's first create a combined 'food points' layer to demonstrate this process. 

In [ ]:
# create identifying attributes
ff['type'] = 'FF'
smkt['type'] = 'S'
# combine supermarkets and fast food restaurants
food = ff.append(smkt)

In [ ]:
print(food['type'].head())
print(food['type'].tail())

The first way to do this is to use `cmap` to assign a pre-designed colour map. The options available can be found here: [https://matplotlib.org/tutorials/colors/colormaps.html](https://matplotlib.org/tutorials/colors/colormaps.html).

Experiment with a few other options.

In [ ]:
# create a point map, specifying colours based on an attribute
food.plot(column='type',     # define the attribute field name
          categorical=True,  # treat it as a qualitative value
          cmap='Set1')   # assign colours

Alternatively, you can manually specify your colours. 
First, create an external dictionary with colours assigned to each food point 'type'. Next, we will map those values based on the data frame column. We will store these colour assignments in the `food` GeoDataFrame for convenience.

In [ ]:
palette = {'FF':'#1b9e77',  # matplotlib can also use HEX colours
           'S':'#7570b3'}

In [ ]:
# use `.map()` to look up the dicionary values based on the food point type
food['cm'] = food['type'].map(palette)
food['cm'].head()

In [ ]:
# create a plot using the manual colour map
# NOTE: assign column of colours to `c`, not `cmap`
food.plot(c=food['cm'])

## Adding external basemaps

So far we've seen how we can create `matplotlib` axes by plotting a GeoDataFrame. We have flexibility to modify the figure size, the colour, symbols and layers in that plot. But our maps of point locations are really lacking context that our readers need. There's no way to (easily) know that we're looking at a map of Southampton.

It's time to expand our maps by adding in some *basemaps* to provide a background for the point data maps.

In [ ]:
import contextily as cx  

To add the basemaps we will be using `contextily` which makes this process very easy! 

Check the documentation here: [https://contextily.readthedocs.io/en/latest/](https://contextily.readthedocs.io/en/latest/). You can add many different kinds of basemaps and even get data from Google Earth Engine.

In [ ]:
# start by creating a basic figure
fig, ax = plt.subplots(figsize=(15,10))

food.plot(ax=ax,
          c=food['cm'] # use our manual color map
         )

# add the basemap to the same axis
# note: this can be a little slow as it has to download tiles
cx.add_basemap(ax, # add to the same plot
               crs=food.crs.to_string()  # set the projection
              )

We now have the default basemap under our food point locations. In the code above, we had to specify the axis and the coordinate reference system. Remember from our earlier practical that most web map tiles are in Web Mercator projection (EPSG:3857). We could have reprojected our food points data to match, but here we're seeing how to modify the tiles. Reprojecting or *warping* the basemap is handled in the background by specifying `crs=`. Note, that you have to convert the GeoDataFrame CRS representation to a string.

The tiles from `contextily` are a form of *raster* or gridded data. We've only worked with vector data so far, but, in fact, we can directly access the basemap raster tiles that are downloaded. The code below opens the imagery with `rasterio`, but the files could also be downloaded an opened with other GIS software.

In [ ]:
# get the extent of the point data
# but we need this in WGS84 to work with contextily
# 'total_bounds' gives us an array of coordinates for bounding box
bb = food.to_crs(epsg=4326).total_bounds

# download tiles to data directory
_ = cx.bounds2raster(*bb,
                     ll=True,
                     path='../data/soton.tif' # save as GeoTiff file
                    )

In [ ]:
# access the raster data
# load new package for raster data
import rasterio
from rasterio.plot import show as rioshow

# create a dataset reader
with rasterio.open("soton.tif") as r:
    rioshow(r)

Experiment with some of the other available basemaps by changing the `source=` to a different provider. The basemap providers that are available by default are coming from a Leaflet extension. Preview them here: [http://leaflet-extras.github.io/leaflet-providers/preview/](http://leaflet-extras.github.io/leaflet-providers/preview/)

Try the satellite imagery ('cx.providers.Esri.WorldImagery'). But sometimes in a map, less is more, a minimalist basemap is more appropriate (such as 'cx.providers.Stamen.TonerLite').

In [ ]:
# Create new plots with food GeoDataFrame and test different basemaps


#

### Adding additional map elements

Our maps are starting to look better. Now we can add basemaps or even satellite imagery to give us a little context for the area we're mapping. But good cartographic design often incorporates a few more details. In this next section we'll add scale bars, to indicate distances, legends to explain the data symbology, and a title. These are all elements you may need to add to publication-ready maps.

#### Scale bar and north arrow

A scale bar is a measured line or patch that depicts distances in the map. Showing distance becomes more difficult when plotting with latitude/longitude since the distance of 1 degree of longitude changes with latitude. But our data are projected (EPSG 27700), so distance is consistently measured in meters in both the x and y direction.

To create a scale bar, we'll add a 'patch' in `matplotlib` of a known size. The north arrow uses similar `matplotlib` techniques with annotations.

In [ ]:
# start by making a base map
fig, ax = plt.subplots(figsize=(15,10))

food.plot(ax=ax,
          c=food['cm'] # use our manual color map
         )

cx.add_basemap(ax=ax,
               crs=food.crs.to_string(),
               source=cx.providers.Stamen.TonerLite)

# add a scale bar patch area
# the scale bar will 3KM in size

# specify the location (in x,y coordinates) and length (all in meters)
x, y, scale_len = 444800, 110200, 3000  

scale_rect = matplotlib.patches.Rectangle((x,y),     # location
                                          scale_len, # length
                                          100,       # height or thickness of scale bar
                                          linewidth=1,
                                          edgecolor='k',facecolor='k' # black in colour
                                         )
# add scale bar onto plot
ax.add_patch(scale_rect)

# add a label to the scale bar
plt.text(x + scale_len/2, y + 400, # locate the label in the center of the bar
         s='3 KM', 
         fontsize=15, 
         horizontalalignment='center')

# add a north arrow
# this assumes that North is up on your map
# if not - change the direction of the arrow
# https://stackoverflow.com/a/58110049/604456

# specify location - in figure proportions (not real-world coordinates)
x, y, arrow_length = 0.7, 0.15, 0.1

ax.annotate('N', #
            xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes)

plt.show()

#### Legend and title

We will now add on to our previous mapping script to add in a legend, with appropriate labels, as well as an informative title. We're building the start of a mapping workflow here: building up a script that allows us to make minor changes and improvements and replicate a plot easily. 

You might also notice that the tiles from `contextily` are no longer slowing you down. That's because the downloaded files have been cached and get reused, making it more efficient to iterate small changes on a plot.

In [ ]:
# start by making a base map
fig, ax = plt.subplots(figsize=(15,10))

food.plot('type',
          ax=ax,
          cmap='Paired', # use a new color map
          # THIS IS NEW
          legend=True, 
          categorical=True,
          legend_kwds={'loc':'lower left',
                       'title':'Food points',
                       'fontsize':16,
                       'title_fontsize':16
                      }
         )

cx.add_basemap(ax=ax,
               crs=food.crs.to_string(),
               source=cx.providers.Stamen.TonerLite)

# add a scale bar patch area
# the scale bar will 3KM in size

# specify the location (in x,y coordinates) and length (all in meters)
x, y, scale_len = 444800, 110200, 3000  

scale_rect = matplotlib.patches.Rectangle((x,y),     # location
                                          scale_len, # length
                                          100,       # height or thickness of scale bar
                                          linewidth=1,
                                          edgecolor='k',facecolor='k' # black in colour
                                         )
# add scale bar onto plot
ax.add_patch(scale_rect)

# add a label to the scale bar
plt.text(x + scale_len/2, y + 400, # locate the label in the center of the bar
         s='3 KM', 
         fontsize=15, 
         horizontalalignment='center')

# add a north arrow
# this assumes that North is up on your map
# if not - change the direction of the arrow
# https://stackoverflow.com/a/58110049/604456

# specify location - in figure proportions (not real-world coordinates)
x, y, arrow_length = 0.7, 0.15, 0.1

ax.annotate('N', #
            xy=(x, y), xytext=(x, y-arrow_length),
            arrowprops=dict(facecolor='black', width=5, headwidth=15),
            ha='center', va='center', fontsize=20,
            xycoords=ax.transAxes)

# add a title
ax.set_title('Southampton food shops', fontdict={'fontsize':18})

plt.show()

For more complex legends, it may be necessary to plot each layer separately and add a `label` so that you can access and set those properties manually. Right now the plot is using the default attributes for the `type` field.

See for an example: [https://www.earthdatascience.org/courses/scientists-guide-to-plotting-data-in-python/plot-spatial-data/customize-vector-plots/python-customize-map-legends-geopandas/](https://www.earthdatascience.org/courses/scientists-guide-to-plotting-data-in-python/plot-spatial-data/customize-vector-plots/python-customize-map-legends-geopandas/).

## Thematic maps

In this next section we will set aside point maps, and return to *choropleth* mapping with our LSOA data with the distance to supermarkets calculated in the last practical. We briefly saw how `cmap` and different classification patterns can be used. We will look at how those break points are created with `mapclassify` in more detail, how to apply them to the continuous data, and adjust the plotting options.

### Load the LSOA data

In [ ]:
lsoa = gpd.read_file('../data/soton_lsoa_distance.gpkg')

In [ ]:
lsoa.head()

### Map classifications

More details on `mapclassify` can be found here: [https://pysal.org/mapclassify/](https://pysal.org/mapclassify/). `GeoPandas` allows us to set the classification 'scheme' directly in the plotting function.

In [ ]:
# create a 7 class plot based on "Natural Breaks"
# use a default colour map pattern
lsoa.plot('smkt_dist',
          scheme='NaturalBreaks',
          k=7,
          legend=True,
          figsize=(15,10)  # notice we can set the size directly here
         )

plt.show()

Remember, if we don't set the `scheme` then the data are treated as a continuous colormap and you'll see a color bar as the legend type.

In [ ]:
lsoa.plot('smkt_dist',
          legend=True,
         )

plt.axis('off') # turn off the "box"
plt.show()

Internally, `mapclassify` is creating a set of breaks for the column of values supplied. We can inspect this directly ourselves.

In [ ]:
# natural breaks
mapclassify.NaturalBreaks(lsoa.smkt_dist, k=7)

Another common classifier is to use quantiles.

In [ ]:
# create quintiles (k=5)
mapclassify.Quantiles(lsoa.smkt_dist, k=5)

Try a few other classifiers on your own - looking at the break points or trying them in a map.

In [ ]:
# test mapclassify options


#

#### Histograms

A good way to inspect your map classification is to view a histogram of the data. This will show you how the underlying data are distributed and how you might visualise them most effectively.

In [ ]:
# plot a basic histogram using `Pandas`
lsoa['smkt_dist'].plot.hist(bins=50)

plt.show()

In [ ]:
# create a classifier and store it to access the bins
# Natural Breaks
mc = mapclassify.NaturalBreaks(lsoa.smkt_dist, k=7)

# plot the historgram
lsoa['smkt_dist'].plot.hist(bins=50)

# add lines for the classification bins
for val in mc.bins:
    plt.axvline(val, 
                color='k',
                linestyle='dashed',
                linewidth=1
               )

plt.show()

Now check the breaks for a quantile classification.

In [ ]:
# create a classifier and store it to access the bins
# Quintiles
mc = mapclassify.Quantiles(lsoa.smkt_dist, k=5)

# plot the historgram
lsoa['smkt_dist'].plot.hist(bins=50)

# add lines for the classification bins
for val in mc.bins:
    plt.axvline(val, 
                color='k',
                linestyle='dashed',
                linewidth=1
               )

plt.show()

### Applying different legends

Let's take what we've learned about reclassifying our data and building legends to start to make a better looking map. Experiment with the code below to make your own map.

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(15,10))

# plot the LSOA data
lsoa.plot('smkt_dist',
          legend=True,
          legend_kwds={'title':'Distance (m)',
                       'title_fontsize':16,
                       'loc':'lower left'
                      },
          scheme='quantiles',
          k=4,
          cmap='Greens',
          edgecolor='k',  # darken the LSOA borders
          ax=ax
         )

# add a title
ax.set_title('Southampton Distance to Supermarkets',
            fontsize=24)
# remove the "box" for a clean look
ax.axis('off')

plt.show()

## Controlling layers

Let's continue to improve on our choropleth map of supermarket distance. We'll add in an overlay of all the supermarket location points. Then we'll see how we can control a few other options for how to symbolise the data.

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(15,10))

# plot the LSOA data
lsoa.plot('smkt_dist',
          legend=True,
          legend_kwds={'title':'Distance (m)',
                       'title_fontsize':16,
                       'loc':'lower left'
                      },
          scheme='quantiles',
          k=4,
          cmap='Greens',
          edgecolor='k',  # darken the LSOA borders
          ax=ax
         )

# THIS IS NEW
pts = plt.scatter(smkt.geometry.x, smkt.geometry.y, 
                  color='blue')

# add a title
ax.set_title('Southampton Distance to Supermarkets',
            fontsize=24)
# remove the "box" for a clean look
ax.axis('off')

plt.show()

## Saving maps

Once we have a created a map we're happy with, let's save it out to a file that we can share or include in a report or presentation.

In [ ]:
# create the figure
fig, ax = plt.subplots(figsize=(15,10))

# plot the LSOA data
lsoa.plot('smkt_dist',
          legend=True,
          legend_kwds={'title':'Distance (m)',
                       'title_fontsize':16,
                       'loc':'lower left'
                      },
          scheme='quantiles',
          k=4,
          cmap='Greens',
          edgecolor='k',  # darken the LSOA borders
          ax=ax
         )

# THIS IS NEW
pts = plt.scatter(smkt.geometry.x, smkt.geometry.y, 
                  color='blue')

# add a title
ax.set_title('Southampton Distance to Supermarkets',
            fontsize=24)
# remove the "box" for a clean look
ax.axis('off')

# SAVE with a DPI of 300
plt.savefig('soton_test.png', dpi=300)

Finished! Well done!